In [1]:
import pandas as pd

In [2]:
base_url = "https://en.wikipedia.org/wiki/"
file = "Resources/forbes_billionaires.csv"

In [3]:
#grab billionaires 
bill_df = pd.read_csv(file)

In [4]:
names = bill_df['Name']
names = list(names)

In [5]:
names = [name.split(' &')[0] for name in names]
names = [name.replace(' ', '+') for name in names]
names[:5]

['Jeff+Bezos', 'Elon+Musk', 'Bernard+Arnault', 'Bill+Gates', 'Mark+Zuckerberg']

In [6]:
len(names)

2755

In [7]:
## trying a different method than just pd.read_html
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import unidecode
from pymongo import MongoClient

In [8]:
base_url = "https://www.google.com/search?q=wikipedia+billionaire+"
executable_path = {'executable_path':ChromeDriverManager().install()}

conn = 'mongodb://localhost:27017'
client = MongoClient(conn)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Get LATEST driver version for 91.0.4472
Trying to download new driver from https://chromedriver.storage.googleapis.com/91.0.4472.101/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\JordanRoessle\.wdm\drivers\chromedriver\win32\91.0.4472.101]


In [9]:
# database setup
db = client.billionaire

In [19]:
%%timeit
list_of_tables = []
for count, name in enumerate(names):
    try:
        browser = Browser('chrome', **executable_path, headless=False)
        browser.visit(base_url + name)
        browser.links.find_by_partial_text('Wikipedia').click()
        soup = BeautifulSoup(browser.html, "html.parser")
        table_html = soup.find("table", class_="infobox biography vcard")
        if not table_html:
            table_html = soup.find("table", class_="infobox vcard")
        df = pd.read_html(str(table_html))[0]
        df.dropna(inplace = True)
        mongo_dict = {'name':name.replace('+', ' ')}
        for index, row in df.iterrows():
            key = unidecode.unidecode(row[0])
            value = unidecode.unidecode(row[1])
            mongo_dict[key] = value
            
        db.wikiTables.insert_one(mongo_dict)
        browser.quit()
    except Exception as e:
        browser.quit()
        print(count, '\n', name, '\n', e)
        

14 
 Ma+Huateng 
 key '.mw-parser-output .nobold{font-weight:normal}Ma Hua Teng ' must not contain '.'


NameError: name 'html_table' is not defined

In [16]:
# # reset database
# db.wikiTables.delete_many({})